# First pass at new raster tutorial material

The target area corresponds to hotosm task 5678 with the following bounding box [104.917171, 16.330147, 105.805508, 15.294198]
https://tasks.hotosm.org/project/5678#bottom

The hotosm task used to kickoff Geohack will cover a different area. Chase Stephens from OSM Seattle (CC'ed) will try to secure a clean task for us (~1 month in advance) which focuses on delineating buildings, as is the focus for task 5678. 

Goals identified by Friederich:
- pull in open source raster data on the fly for the specified bounding box, at a resolution sufficient to delineate buildings
- learn how to explore the data using rasterio and gdal etc.
- groom and clean - ready for ML (could be as simple as changing the projection and dropping unnecessary bands, for example)
- if open raster data sources aren’t at a high enough resolution - good discussion point - can stage a dataset that is. this would also teach folks how to add in their own data and coregister it into the same coordinate system and projection.

In [9]:
bbox = [104.917171, 16.330147, 105.805508, 15.294198]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

## 1) Look at a leaflet slippy map to get accustomed to area of interest

In [10]:
import ipyleaflet
from ipyleaflet import Map, Rectangle
m = Map(center=bbox_ctr, zoom=6)
rectangle = Rectangle(bounds=((south, west), (north, east))) #SW and NE corners of the rectangle (lat, lon)
m.add_layer(rectangle)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## 2) NASA GIBS provides free imagery tiles that can be easily displayed with ipyleaflet

The NASA Worldview web application is a way to explore all GIBS datasets https://worldview.earthdata.nasa.gov/
More here: https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/gibs

NOTES: 
* add some information about MODIS. Coarse resolution, not optimal for building detection, but daily and free!
* also, odds are there are clouds obstructing your view of earth's surface!
* GIBS has pre-rendered images good for visualizations, if you're interested in analytic values for scientific analysis, this isn't the best option

In [14]:
from ipyleaflet import basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

m = Map(center=bbox_ctr, zoom=6)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2019-06-30")
left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m.add_layer(rectangle)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## 3) Sat-search is a tool to discover free imagery on AWS 

https://registry.opendata.aws/
    
    
NOTES: 
* add a table comparing Landsat8, Sentinel2, CBERS (resolution/posting, repeat interval)

In [17]:
import satsearch

In [20]:
# Custom syntax (additional fields, query strings instead of query dict)
properties =  ["landsat:tier=T1"] 
bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)
results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

447 items


In [22]:
# Save for later/sharing with others
from satstac import Items

items = results.items()
items.save('items.json')
items = Items.load('items.json')
#items.bbox()

In [23]:
import geopandas as gpd

# Load results to pandas geodataframe
gf = gpd.read_file('items.json')
gf = gf.sort_values('datetime').reset_index(drop=True)
print('records:', len(gf))
gf.head()

records: 447


,id,collection,datetime,eo:sun_azimuth,eo:sun_elevation,eo:cloud_cover,eo:row,eo:column,landsat:product_id,landsat:scene_id,landsat:processing_level,landsat:tier,eo:epsg,eo:instrument,eo:off_nadir,eo:platform,eo:bands,eo:gsd,landsat:revision,geometry
0,LC81270482013093LGN02,landsat-8-l1,2013-04-03T03:25:50,113.391559,62.940947,33,048,127,LC08_L1TP_127048_20130403_20170505_01_T1,LC81270482013093LGN02,L1TP,T1,32648,OLI_TIRS,0,landsat-8,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",15,None,"POLYGON ((103.9001025239973 18.33384943378347,..."
1,LC81270492013093LGN02,landsat-8-l1,2013-04-03T03:26:14,110.525644,63.279740,24,049,127,LC08_L1TP_127049_20130403_20170505_01_T1,LC81270492013093LGN02,L1TP,T1,32648,OLI_TIRS,0,landsat-8,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",15,None,"POLYGON ((103.5790674890864 16.89265606069266,..."
2,LC81260492013098LGN02,landsat-8-l1,2013-04-08T03:22:47,106.960273,64.409286,99,049,126,LC08_L1TP_126049_20130408_20170505_01_T1,LC81260492013098LGN02,L1TP,T1,32648,OLI_TIRS,0,landsat-8,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",15,None,"POLYGON ((104.4758565819867 16.88532916228322,..."
3,LC81260492013105LGN03,landsat-8-l1,2013-04-15T03:20:39,101.611233,65.716617,15,049,126,LC08_L1TP_126049_20130415_20180523_01_T1,LC81260492013105LGN03,L1TP,T1,32648,OLI_TIRS,0,landsat-8,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",15,None,"POLYGON ((105.0321488643234 16.88585447649133,..."
4,LC81260502013105LGN03,landsat-8-l1,2013-04-15T03:21:03,98.314642,65.754833,54,050,126,LC08_L1TP_126050_20130415_20180523_01_T1,LC81260502013105LGN03,L1TP,T1,32648,OLI_TIRS,0,landsat-8,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",15,None,"POLYGON ((104.7144716209946 15.44129277521946,..."


In [24]:
# Plot search AOI and frames on a map
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

footprints = gf.loc[:,('id','geometry')].hvplot(geo=True)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) #.redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * labels

ModuleNotFoundError: No module named 'hvplot'